In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')


In [ ]:
import os
import torch
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image

# Path to your dataset in Google Drive
dataset_dir = '/content/drive/MyDrive/jenita'


In [ ]:
class ImageDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []

        # Load image paths and labels
        for label_dir in os.listdir(root_dir):
            label_dir_path = os.path.join(root_dir, label_dir)
            if os.path.isdir(label_dir_path):
                for img_file in os.listdir(label_dir_path):
                    if img_file.endswith(('.jpg', '.png')):
                        self.image_paths.append(os.path.join(label_dir_path, img_file))
                        self.labels.append(0 if label_dir == 'benign' else 1)  # Adjust labels accordingly

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        label = self.labels[idx]

        img_pil = Image.open(img_path).convert('RGB')

        if self.transform:
            img_pil = self.transform(img_pil)

        return img_pil, label


In [ ]:
# Transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Load the dataset
dataset = ImageDataset(root_dir=dataset_dir, transform=transform)
train_loader = DataLoader(dataset, batch_size=20, shuffle=True)


In [ ]:
import torch.nn as nn
import torch.optim as optim
from torchvision import models

# Define the model (MobileNetV2)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.mobilenet_v2(weights=None)  # Use `weights=None` instead of `pretrained=False`
model.classifier[1] = nn.Linear(model.last_channel, 2)  # Adjust for the number of classes (2 for binary classification)
model = model.to(device)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [ ]:
n_epochs = 10
train_loss = []
train_accuracy = []

for epoch in range(n_epochs):
    model.train()
    correct = 0
    total = 0
    running_loss = 0.0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_loader)
    epoch_acc = correct / total

    train_loss.append(epoch_loss)
    train_accuracy.append(epoch_acc)

    print(f'Epoch [{epoch+1}/{n_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.4f}')


In [ ]:
# Save the model
torch.save(model.state_dict(), '/content/Tumor_Detection_MobileNetV2.pt')


In [ ]:
import matplotlib.pyplot as plt

# Plot accuracy and loss
def plot_acc_loss(train_loss, train_accuracy):
    epochs = range(1, len(train_loss) + 1)

    plt.figure()
    plt.plot(epochs, train_loss, 'b', label='Training loss')
    plt.title('Training loss')
    plt.legend()

    plt.figure()
    plt.plot(epochs, train_accuracy, 'b', label='Training accuracy')
    plt.title('Training accuracy')
    plt.legend()

    plt.show()

plot_acc_loss(train_loss, train_accuracy)


In [ ]:
# Load the trained model's weights
model.load_state_dict(torch.load('/content/Tumor_Detection_MobileNetV2.pt'))
model.eval()  # Set the model to evaluation mode

# Define the image preprocessing steps for inference
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

def load_image(image_path):
    """Load and preprocess the image."""
    img_pil = Image.open(image_path).convert('RGB')
    image = preprocess(img_pil)
    image = image.unsqueeze(0)  # Add batch dimension
    return image.to(device)

def predict(image_path, model):
    """Predicts the class of an image."""
    image = load_image(image_path)
    with torch.no_grad():
        outputs = model(image)
        _, predicted = torch.max(outputs.data, 1)
    return predicted.item()


In [ ]:
# Get class names from directory structure
class_names = ['benign', 'malignant']  # Adjust according to your labels

image_path = '/content/drive/MyDrive/jenita/some_image.jpg'  # Replace with your image path
predicted_class = predict(image_path, model)
predicted_class_name = class_names[predicted_class]

print(f'Predicted Class: {predicted_class_name}')
